### Imports

In [2]:
import numpy as np
import os
import os.path
import pandas as pd
import cv2
import tensorflow as tf
import pickle
from PIL import Image, ImageDraw

In [29]:
list_tasks = pickle.load( open( "/home/acarlier/code/OrnithoScope/data/pickles/list_tasks.p", "rb" ) )
model_path = '/home/acarlier/code/OrnithoScope/saved_models/model_efficientDet0.tflite'
input_csv_path = '/home/acarlier/code/OrnithoScope/data/csv/input.csv'
pickle_tasks_path = '/home/acarlier/code/OrnithoScope/data/pickles/tasks/'
columns_names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"]
#output_images_directory = '/home/acarlier/code/data_ornitho/output_images/'
#comparaison_images_directory = '/home/acarlier/code/data_ornitho/comparaison_verite_terrain/comparaison_efficientDet1/'


### Load tasks df

In [44]:
def load_df_per_tasks(pickle_tasks_path):
    dfs = []
    pickle_tasks_files = os.listdir(pickle_tasks_path)
    for file in pickle_tasks_files:
        dfs.append(pickle.load( open( f"{pickle_tasks_path}{file}", "rb" ) ))
    return dfs

In [45]:
dfs = load_df_per_tasks(pickle_tasks_path)

In [66]:
def get_classes_from_input():
    input = pd.read_csv('/home/acarlier/code/OrnithoScope/data/csv/input.csv', names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"])
    return input["label"].unique().tolist()

In [64]:
classes = get_classes_from_input()
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

In [67]:
def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  #print(tf.shape(resized_img))
  return resized_img, original_image

def preprocess_images(image_paths, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_paths[0])
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  
  resized_imgs = resized_img
  
  for i in range(1,len(image_paths)):
    img = tf.io.read_file(image_paths[i])
    img = tf.io.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    resized_img = tf.image.resize(img, input_size)
    resized_img = resized_img[tf.newaxis, :]
    resized_img = tf.cast(resized_img, dtype=tf.uint8)
    
    resized_imgs = tf.concat([resized_imgs, resized_img], axis=0)
    #print(tf.shape(resized_imgs))
    
  return resized_imgs


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()
  #print("avant modèle")
  # Feed the input image to the model
  output = signature_fn(images=image)
  #interpreter.set_tensor(0, image)
  #interpreter.invoke()
  #print("après modèle")
  
  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
  print(input_height, input_width)
  #print("interpreter")
  
  # Load the input image and preprocess it
  preprocessed_image, _ = preprocess_image(
      image_path,
      (input_height, input_width)
    )
  #print("preprocessing")
  
  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)
  
  return results

In [68]:
def get_test_images(df):
    list_image_test = df["file_path"].unique().tolist() # return all unique file_path to test, no prediction on two same image
    return list_image_test
loi = get_test_images(dfs[0])
print(loi[0])
print(len(loi))

/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_20210611_Lab/2021-06-11-13-44-43.jpg
411


In [74]:
# get predictions from model
def prediction_test_dataset(df):
    results = []
    list_of_images = get_test_images(df)
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    #list_of_images = ['/home/acarlier/code/test/2021-06-11-13-44-43.jpg']
    # Load the TFLite model
    print(len(list_of_images),' to predict')
    for img in list_of_images[:2]:
        if list_of_images.index(img)%100 == 0:
            print(list_of_images.index(img))
        img_name = img.split('/')[-1]
        # file_path = '{}{}'.format(output_images_directory, img_name)
        # if os.path.exists(file_path):
        #     list_of_images.remove(img)
        #     #print(len(list_of_images),' remaining')
        # else:
        INPUT_IMAGE_URL = img #@param {type:"string"}
        DETECTION_THRESHOLD = 0.3 #@param {type:"number"}
        TEMP_FILE = img
        FILE_NAME = TEMP_FILE.split("/")[-1]
        #print('test1.2 : ok')
        #!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
        #!wget -q -O $INPUT_IMAGE_URL
        #print('test1.3 : ok')
        try:
            with open(TEMP_FILE) as file:
                #im = Image.open(TEMP_FILE)
                #im.thumbnail((512, 512), Image.ANTIALIAS)
                #im.save(TEMP_FILE, 'JPEG')
                # Run inference and draw detection result on the local copy of the original file
                result = run_odt_and_draw_results(
                    TEMP_FILE,
                    interpreter,
                    threshold=DETECTION_THRESHOLD
                )
                results.append(result)
                # Show the detection result
                #Image.fromarray(detection_result_image)
                #result = Image.fromarray(detection_result_image)
                #result.save("{}{}".format(output_images_directory, FILE_NAME), 'JPEG')
                #result.show()
        except OSError as e:
            print(e.errno)
    return results

In [ ]:
# generate prediction results with pickles
# results_predictions = prediction_test_dataset()
# pickle.dump(results_predictions, open( "/home/acarlier/code/OrnithoScope/saved_models/results_EfficientDet0.p", "wb" ) )

# # load prediction results with pickles 
# #results_predictions = pickle.load( open( "/home/acarlier/code/OrnithoScope/saved_models/results_EfficientDet1.p", "rb" ) )


# classes = get_classes_from_input()
# print(len(results_predictions))
# print(classes)
# results_predictions[0]

In [75]:
for df in dfs:
    task_name = df['task_name'].unique()[0]
    df = df.loc[:, ~df.columns.isin(['task_name', 'Site', 'Day','Day2'])]
    results_predictions = prediction_test_dataset(df)
    pickle.dump(results_predictions, open( f"/home/acarlier/code/OrnithoScope/saved_models/tasks/{task_name}.p", "wb" ) )

411  to predict
0
320 320
320 320
328  to predict
0
320 320
320 320
400  to predict
0
320 320
320 320
805  to predict
0
320 320
320 320
196  to predict
0
320 320
320 320
169  to predict
0
320 320
320 320
228  to predict
0
320 320
320 320
979  to predict
0
320 320
320 320
